# Avoiding Fish Fries at All Cost
### With Haiden Hunter


Datasets: [Pittsburgh Fish Fry Locations](https://data.wprdc.org/dataset/pittsburgh-fish-fry-map)

![alt text](https://media.giphy.com/media/v1.Y2lkPTc5MGI3NjExeHk2eDcxdWIxbDNpM3RtdGl2emd5eHdxNjZqbTJkNHhybzZqeWs1aCZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/13U2NlgRx3QvMQ/giphy.gif)

In [11]:
# Ok... I decided to throw every possible import I could need here.
# I'd rather have it and not use it all that much then forget about it and never use it.
import fpsnippets
import pandas as pd
import numpy as np
import geopandas as gpd

#Lets load in all of the fish fry data from 2019 to 2023 and merge them into one dataset to clean up.
fishFryData2023 = pd.read_csv("https://data.wprdc.org/datastore/dump/511a29f6-3217-4f61-a9ba-b3b5b35ab5fb")
fishFryData2022 = pd.read_csv("https://data.wprdc.org/dataset/682daad1-6d3a-45d3-8710-6c961146e19b/resource/f4d7e81a-ac39-4f84-a249-c68524e8258a/download/2022_pittsburgh_fish_fry_locations.csv")
fishFryData2021 = pd.read_csv("https://data.wprdc.org/dataset/682daad1-6d3a-45d3-8710-6c961146e19b/resource/dfa58a5b-d221-411f-bd7e-32837ff99993/download/2021_pittsburgh_fish_fry_locations.csv")
fishFryData2020 = pd.read_csv("https://data.wprdc.org/dataset/682daad1-6d3a-45d3-8710-6c961146e19b/resource/d802d628-bd44-47cc-bbc9-c691f9026ca1/download/2020_pittsburgh_fish_fry_locations.csv")
fishFryData2019 = pd.read_csv("https://data.wprdc.org/dataset/682daad1-6d3a-45d3-8710-6c961146e19b/resource/5b58c467-8e6a-4abc-9dd5-a39881770b3c/download/2019_pittsburgh_fish_fry_locations.csv")

# Merging data.
fishFryData = pd.concat([fishFryData2023, fishFryData2022, fishFryData2021, fishFryData2020, fishFryData2019]).reset_index(drop=True)
fishFryData.head(3)

,_id,validated,venue_name,venue_type,venue_address,website,events,etc,menu_url,menu_text,...,email,homemade_pierogies,take_out,alcohol,lunch,handicap,publish,id,latitude,longitude
0,8680.0,t,"St. Peter Church, St. Faustina Parish",Church,"670 South Main Street, Slippery Rock, Pennsylv...",https://stfaustinaparish.org/,"Friday Feb 24 from 4:00 PM to 7:00 PM, Friday ...",NaN,https://stfaustinaparish.org/,"$12 per meal, \nALL YOU CAN EAT $18\nKids unde...",...,NaN,NaN,t,NaN,f,NaN,t,NaN,41.056430,-80.050690
1,8681.0,f,Hartford Heights Volunteer Fire Department,Fire Department,"14335 U.S. 30, Irwin, PA 15642",https://www.facebook.com/Hartford-Heights-VFD-...,"Friday Feb 24 from 4:00 PM to 8:00 PM, Friday ...",NaN,NaN,NaN,...,NaN,f,t,NaN,f,NaN,f,NaN,40.352969,-79.767295
2,8682.0,t,"Our Lady of Joy Church, Holy Family Parish",Church,"2000 O'Block Road Holiday Park, PA",https://holyfamilypgh.org/,"Wednesday Feb 22 from 4:00 PM to 7:00 PM, Frid...",NaN,chrome-extension://efaidnbmnnnibpcajpcglclefin...,"Hand breaded fish dinner $14 includes fries, s...",...,NaN,NaN,t,NaN,f,NaN,t,NaN,40.480007,-79.719563


#### Why not delete duplicates?

At first I was planning on removing them. As when compiling multiple years of data, I was worried about the clutter caused by reoccuring fish fries. However, I realized that it may be benefitial to keep the reoccuring fish fries. As it verifies whether or not it was a one off fish fry or a community staple.

In [12]:
# Removing nonessential columns.
fishFryData = fishFryData[['validated', 'venue_name', 'latitude', 'longitude']]
fishFryData.head(3)

,validated,venue_name,latitude,longitude
0,t,"St. Peter Church, St. Faustina Parish",41.056430,-80.050690
1,f,Hartford Heights Volunteer Fire Department,40.352969,-79.767295
2,t,"Our Lady of Joy Church, Holy Family Parish",40.480007,-79.719563


In [35]:
# Sorting through valid Pittsburgh neighborhoods.
cleanFishFryData = pd.DataFrame(columns=['Neighborhood'])
for x, row in fishFryData.iterrows():
    neighborhood = fpsnippets.geo_to_neighborhood(row['latitude'], row['longitude'])
    if neighborhood != None:
        if ((row["validated"]) == "t") or ((row["validated"]) == True):
            print(neighborhood)
            pd.concat(cleanFishFryData, {'Neighborhood' : neighborhood})

Central Lawrenceville


<ipython-input-35-3e5191c0c514>:8: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  pd.concat(cleanFishFryData, {'Neighborhood' : neighborhood})


TypeError: first argument must be an iterable of pandas objects, you passed an object of type "DataFrame"

In [14]:
rawFishFryNeighborhoods = gpd.read_file("Neighborhoods/Neighborhoods_.shp")

In [15]:
fishFryNeighborhoods = pd.DataFrame(columns = ["Neighborhood", "FishFrys"])